# Installation

In [ ]:
!python -m pip install paddlepaddle==3.2.0 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/
!python -m pip install "paddleocr[all]"

Looking in indexes: https://www.paddlepaddle.org.cn/packages/stable/cpu/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: opt_einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


# Text Recognition

In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR (
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False) # text detection + text recognition

/usr/local/lib/python3.12/dist-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in `/root/.paddlex/official_models/PP-OCRv5_server_det`.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

inference.yml:   0%|          | 0.00/903 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

inference.json: 0.00B [00:00, ?B/s]

inference.yml: 0.00B [00:00, ?B/s]

inference.pdiparams:   0%|          | 0.00/84.4M [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

In [ ]:
result = ocr.predict ("receipt_1.jpg")
for res in result:
    res.print()
    res.save_to_img("output")
    res.save_to_json("output")

{'res': {'input_path': 'receipt_1.jpg', 'page_index': None, 'model_settings': {'use_doc_preprocessor': False, 'use_textline_orientation': False}, 'dt_polys': array([[[ 198,  108],
        ...,
        [ 198,  158]],

       ...,

       [[2086, 2185],
        ...,
        [2086, 2223]]], dtype=int16), 'text_det_params': {'limit_side_len': 64, 'limit_type': 'min', 'thresh': 0.3, 'max_side_limit': 4000, 'box_thresh': 0.6, 'unclip_ratio': 1.5}, 'text_type': 'general', 'textline_orientation_angles': array([-1, ..., -1]), 'text_rec_score_thresh': 0.0, 'return_word_box': False, 'rec_texts': ['Invoice no:25698672', 'Date of issue:', '08/01/2015', 'Seller:', 'Client:', 'Gordon, Alvarado and Jones', 'Manning-Ho', '943SeanParkSuite284', '92026Michael Terrace Apt.138', 'NorthDerrick,OR67536', 'WestMichael,IL42693', 'TaxId:936-79-2794', 'TaxId:960-85-1578', 'IBAN:GB47JKAT88391563723291', 'ITEMS', 'No.', 'Description', 'Qty', 'UM', 'Net price', 'Networth', 'VAT [%]', 'Gross', 'worth', '1.', 'CCNA20

# **LLM Parsing**

## Installation

In [ ]:
%%capture
import os, re, json

# Do this only in Colab notebooks! Otherwise use pip install unsloth
import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2


## Data Preparation

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
dataset = load_dataset("mychen76/invoices-and-receipts_ocr_v1")
dataset

README.md:   0%|          | 0.00/782 [00:00<?, ?B/s]

data/train-00000-of-00001-76ffc8319f74dd(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/test-00000-of-00001-af2d92d1cee2851(…):   0%|          | 0.00/18.8M [00:00<?, ?B/s]

data/valid-00000-of-00001-894b4e1f736b57(…):   0%|          | 0.00/14.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/125 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/70 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 2043
    })
    test: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 125
    })
    valid: Dataset({
        features: ['image', 'id', 'parsed_data', 'raw_data'],
        num_rows: 70
    })
})

In [ ]:
sample=dataset['train'][0]
print(sample)

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2481x3508 at 0x7BC1EAF3ED50>, 'id': '0', 'parsed_data': '{"xml": "", "json": "{\'header\': {\'invoice_no\': \'40378170\', \'invoice_date\': \'10/15/2012\', \'seller\': \'Patel, Thompson and Montgomery 356 Kyle Vista New James, MA 46228\', \'client\': \'Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601\', \'seller_tax_id\': \'958-74-3511\', \'client_tax_id\': \'998-87-7723\', \'iban\': \'GB77WRBQ31965128414006\'}, \'items\': [{\'item_desc\': \\"Leed\'s Wine Companion Bottle Corkscrew Opener Gift Box Set with Foil Cutter\\", \'item_qty\': \'1,00\', \'item_net_price\': \'7,50\', \'item_net_worth\': \'7,50\', \'item_vat\': \'10%\', \'item_gross_worth\': \'8,25\'}], \'summary\': {\'total_net_worth\': \'$7,50\', \'total_vat\': \'$0,75\', \'total_gross_worth\': \'$8,25\'}}", "kie": ""}', 'raw_data': '{"ocr_words": "[\'Invoice no: 40378170\', \'Date of issue:\', \'10/15/2012\', \'Seller:\', \'Client:\

In [ ]:
sample["raw_data"]

'{"ocr_words": "[\'Invoice no: 40378170\', \'Date of issue:\', \'10/15/2012\', \'Seller:\', \'Client:\', \'Patel, Thompson and Montgomery\', \'Jackson, Odonnell and Jackson.\', \'356 Kyle Vista\', \'267 John Track Suite 841\', \'New James, MA 46228\', \'Jenniferville, PA 98601\', \'Tax Id: 958-74-3511\', \'Tax Id: 998-87-7723\', \'IBAN: GB77WRBQ31965128414006\', \'ITEMS\', \'UM\', \'No.\', \'Description\', \'Qty\', \'Net price\', \'Net worth\', \'VAT [%]\', \'Gross\', \'worth\', \\" Leed\'s Wine Companion Bottle\\", \'1,00\', \'each\', \'7,50\', \'7,50\', \'10%\', \'8,25\', \'Corkscrew Opener Gift Box Set\', \'with Foil Cutter\', \'SUMMARY\', \'VAT [%]\', \'VAT\', \'Net worth\', \'Gross worth\', \'7,50\', \'10%\', \'0,75\', \'8,25\', \'Total\', \'$7,50\', \'$ 0,75\', \'$8,25\']", "ocr_boxes": "[[[[196.0, 110.0], [801.0, 110.0], [801.0, 161.0], [196.0, 161.0]], (\'Invoice no: 40378170\', 0.9985853433609009)], [[[196.0, 212.0], [517.0, 212.0], [517.0, 259.0], [196.0, 259.0]], (\'Date of 

In [ ]:
sample["parsed_data"]

'{"xml": "", "json": "{\'header\': {\'invoice_no\': \'40378170\', \'invoice_date\': \'10/15/2012\', \'seller\': \'Patel, Thompson and Montgomery 356 Kyle Vista New James, MA 46228\', \'client\': \'Jackson, Odonnell and Jackson 267 John Track Suite 841 Jenniferville, PA 98601\', \'seller_tax_id\': \'958-74-3511\', \'client_tax_id\': \'998-87-7723\', \'iban\': \'GB77WRBQ31965128414006\'}, \'items\': [{\'item_desc\': \\"Leed\'s Wine Companion Bottle Corkscrew Opener Gift Box Set with Foil Cutter\\", \'item_qty\': \'1,00\', \'item_net_price\': \'7,50\', \'item_net_worth\': \'7,50\', \'item_vat\': \'10%\', \'item_gross_worth\': \'8,25\'}], \'summary\': {\'total_net_worth\': \'$7,50\', \'total_vat\': \'$0,75\', \'total_gross_worth\': \'$8,25\'}}", "kie": ""}'

In [ ]:
max_seq_length = 7168

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
system_message = """You are a POS expert, and data engineer. with many years on working with
complex receipts and invoices, with different structures. I need you to parse, detect, recognize
 and convert the following receipts and invoices, from the raw ocr image outputs into structured receipt format.
 the output must be a well-formed json object"""

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
def formatting_prompts(examples):
  texts = []
  for i in range(len(examples["raw_data"])):
    raw_data_item = json.loads(examples["raw_data"][i])["ocr_words"]
    parsed_data_item = json.loads(examples["parsed_data"][i])['json']

    convo = [{"role": "system", "content":system_message},
            {"role": "user", "content": raw_data_item},
            {"role": "assistant", "content": parsed_data_item}
            ]
    texts.append(tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False))

  return {"text":texts}

In [ ]:
formatting_prompts({"raw_data": [sample["raw_data"]], "parsed_data": [sample["parsed_data"]]})

{'text': ['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a POS expert, and data engineer. with many years on working with \ncomplex receipts and invoices, with different structures. I need you to parse, detect, recognize\n and convert the following receipts and invoices, from the raw ocr image outputs into structured receipt format.\n the output must be a well-formed json object<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n[\'Invoice no: 40378170\', \'Date of issue:\', \'10/15/2012\', \'Seller:\', \'Client:\', \'Patel, Thompson and Montgomery\', \'Jackson, Odonnell and Jackson.\', \'356 Kyle Vista\', \'267 John Track Suite 841\', \'New James, MA 46228\', \'Jenniferville, PA 98601\', \'Tax Id: 958-74-3511\', \'Tax Id: 998-87-7723\', \'IBAN: GB77WRBQ31965128414006\', \'ITEMS\', \'UM\', \'No.\', \'Description\', \'Qty\', \'Net price\', \'Net worth\', \'VAT [%]\', \'Gross\', \'worth\', " Lee

In [ ]:
standardized_dataset = dataset.map(formatting_prompts, batched=True, remove_columns=['image', 'id', 'parsed_data', 'raw_data'])
standardized_dataset["train"][0]["text"]

Map:   0%|          | 0/2043 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a POS expert, and data engineer. with many years on working with \ncomplex receipts and invoices, with different structures. I need you to parse, detect, recognize\n and convert the following receipts and invoices, from the raw ocr image outputs into structured receipt format.\n the output must be a well-formed json object<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n[\'Invoice no: 40378170\', \'Date of issue:\', \'10/15/2012\', \'Seller:\', \'Client:\', \'Patel, Thompson and Montgomery\', \'Jackson, Odonnell and Jackson.\', \'356 Kyle Vista\', \'267 John Track Suite 841\', \'New James, MA 46228\', \'Jenniferville, PA 98601\', \'Tax Id: 958-74-3511\', \'Tax Id: 998-87-7723\', \'IBAN: GB77WRBQ31965128414006\', \'ITEMS\', \'UM\', \'No.\', \'Description\', \'Qty\', \'Net price\', \'Net worth\', \'VAT [%]\', \'Gross\', \'worth\', " Leed\'s Wine 

## Modeling and Training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

Unsloth 2025.11.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = standardized_dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        max_steps = -1, # Set to -1 when num_train_epochs is set
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
        push_to_hub=True,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2043 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,043 | Num Epochs = 3 | Total steps = 768
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.616100
2,2.801100
3,2.620700
4,2.929200
5,2.057900
6,2.288600
7,2.264200
8,2.295800
9,1.892700
10,1.957300


## Testing

In [ ]:
detections = """{"ocr_words": "['Invoice no: 25698672', 'Date of issue:.', '08/01/2015', 'Seller:', 'Client:', 'Gordon, Alvarado and Jones.', 'Manning-Ho', '943 Sean Park Suite 284', '92026 Michael Terrace Apt. 138', 'North Derrick, OR 67536', 'West Michael, IL 42693', 'Tax Id: 936-79-2794', 'Tax Id: 960-85-1578', 'IBAN: GB47JKAT88391563723291', 'ITEMS', 'No.', 'Description', 'Qty', 'UM', 'Net price', 'Net worth', 'VAT [%]', 'Gross', ' worth', 'CCNA 200-301 50GB AUG 2020', '5,00', 'each', '1,99', '9,95', '10%', '10,94', 'Cisco Video/Official Books/Exam', 'Dumps/Licences', 'Tales from the Buffalo Bills', '5,00', '5,50', '27,50', '10%', '30,25', '2.', 'each', 'A boy who grew up eating', '3.', '4,00', 'each', '19,50', '78,00', '10%', '85,80', \"nightmare It's Okay to Not Be\", 'Okay Picturebook ', 'SUMMARY', 'VAT', 'VAT [%]', 'Net worth', 'Gross worth', '10%', '115,45', '11,55', '127,00', 'Total', '$ 115,45', '$ 11,55', '$ 127,00']" }"""

In [ ]:

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [

            {"role": "system", "content":system_message},
            {"role": "user", "content": detections},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a POS expert, and data engineer. with many years on working with \ncomplex receipts and invoices, with different structures. I need you to parse, detect, recognize\n and convert the following receipts and invoices, from the raw ocr image outputs into structured receipt format.\n the output must be a well-formed json object<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{"ocr_words": "[\'Invoice no: 25698672\', \'Date of issue:.\', \'08/01/2015\', \'Seller:\', \'Client:\', \'Gordon, Alvarado and Jones.\', \'Manning-Ho\', \'943 Sean Park Suite 284\', \'92026 Michael Terrace Apt. 138\', \'North Derrick, OR 67536\', \'West Michael, IL 42693\', \'Tax Id: 936-79-2794\', \'Tax Id: 960-85-1578\', \'IBAN: GB47JKAT88391563723291\', \'ITEMS\', \'No.\', \'Description\', \'Qty\', \'UM\', \'Net price\', \'Net worth\', \'VAT [%]\', \'Gross\',\'worth\', \'CC

## Saving and Loading

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")
model.push_to_hub("Folefac/llama-3.2-3B-ocr", ) # Online saving
tokenizer.push_to_hub("Folefac/llama-3.2-3B-ocr",) # Online saving

README.md:   0%|          | 0.00/612 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  43%|####3     | 41.9MB / 97.3MB            

Saved model to https://huggingface.co/Folefac/llama-3.2-3B-ocr


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpsojnt11d/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

In [ ]:
from transformers import TextStreamer

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Folefac/ocr-llama-3.2-3b", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference



In [ ]:

messages = [

            {"role": "system", "content":system_message},
            {"role": "user", "content": detections},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")


In [ ]:

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.0, min_p = 0.1)

{"invoice_no": "25698672", "invoice_date": "08/01/2015", "seller": "Gordon, Alvarado and Jones 943 Sean Park Suite 284 North Derrick, OR 67536", "client": "Manning-Ho 92026 Michael Terrace Apt. 138 West Michael, IL 42693", "seller_tax_id": "936-79-2794", "client_tax_id": "960-85-1578", "iban": "GB47JKAT88391563723291", "item_desc": "Tales from the Buffalo Bills A boy who grew up eating nightmare It's Okay to Not Be Okay Picturebook", "item_qty": "4,00", "item_net_price": "5,50", "item_net_worth": "22,00", "item_vat": "10%", "item_gross_worth": "24,20", "total_net_worth": "$ 115,45", "total_vat": "$11,55", "total_gross_worth": "$ 127,00"}<|eot_id|>
